In [33]:
import chromadb
from dotenv import load_dotenv


True

In [53]:
import os
import requests
from chromadb import Documents, Embeddings, EmbeddingFunction
from typing import  Union, TypeVar 

load_dotenv()

print(os.getenv("HF_API_KEY"))
print(os.getenv("EMBEDDED_ENDPOINT"))

Embeddable = Union[Documents, None]
D = TypeVar("D", bound=Embeddable, contravariant=True)

class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, api_key="", enpoint_url=""):
        self._api_key = api_key
        self._endpoint_url = enpoint_url
        self._session = requests.Session()

    def encode(self, input: D) -> Embeddings:
        # Send the input to the embedding endpoint and get the embeddings
        response = self._session.post(  # type: ignore
            self._endpoint_url,
            json={
                "inputs": input,
                "parameters": {"wait_for_model": "true"}},
            headers={"authorization": f"Bearer {self._api_key}"}
        )
        response.raise_for_status()  # Raise an exception for HTTP errors
        embeddings = self.session.json()  # Assuming the endpoint returns a JSON array of embeddings
        return embeddings

ef = MyEmbeddingFunction(api_key=os.getenv("HF_API_KEY"), enpoint_url=os.getenv("EMBEDDED_ENDPOINT"))

print(ef.encode(["Hello", "World"]))



HTTPError: 401 Client Error: Unauthorized for url: https://c9ejquzh6yum3xqf.us-east-1.aws.endpoints.huggingface.cloud

In [3]:
client = chromadb.PersistentClient(path="./db")

In [18]:
import os

collection = client.get_or_create_collection(name="chromadb_demo", embedding_function=MyEmbeddingFunction(os.getenv("EMBEDDED_ENDPOINT")))

In [19]:
poetry = [
    "Birds fly over the rainbow",
    "Why do birds suddenly appear",
    "A fast brown animal leaps over a sleepy canine",
    "The lazy dog lies under the shady tree",
    "The quick brown fox jumps over the lazy dog",
]

historical = [
    "Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections.", 
    "Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service.",
    "One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court." ,
    "And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence."
]

In [36]:
import requests

load_dotenv()

api_url = os.getenv("EMBEDDED_ENDPOINT")
api_key = os.getenv("HF_API_KEY")

req_header = {"Authorization": f"Bearer {api_key}"}

req_body = {"texts": poetry[0]}

response = requests.post(
    url=api_url,
    headers=req_header,
    json={
        "inputs": poetry[0],
        "parameters": {
            
        }
    }
)

print(response.json())

{'error': '401 Unauthorized'}


In [40]:
import uuid

# Add poem to collection
collection.add(
    documents=[s for s in poetry],
    metadatas=[{"type": "poetry"} for _ in range(len(poetry))],
    ids=[str(uuid.uuid4()) for _ in range(len(poetry))]
)

HTTPError: 401 Client Error: Unauthorized for url: https://c9ejquzh6yum3xqf.us-east-1.aws.endpoints.huggingface.cloud